### The main tasks:
- Check outliers
- Explore the distribution of 
- Explore relationships
- Check seasonality

## 0. Import data

In [1]:
from sqlalchemy import create_engine, URL, text
from dotenv import load_dotenv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
url = URL.create(
  "postgresql+psycopg2",
  username='admin',
  database='forex',
  password='pass',
  host='localhost',
  port='2345'
)
engine = create_engine(url)

In [2]:
load_dotenv()

pg_user = os.environ["POSTGRES_USER"]
pg_db = os.environ["POSTGRES_DB"]

url = URL.create(
  "postgresql+psycopg2",
  username=pg_user,
  database=pg_db
)

engine = create_engine(url)

## 1. Outliers